In [ ]:
# MACD crossover strategy for the S&P 500 ETF

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pandas'

In [2]:
SHORT_EMA_PERIOD = 12
LONG_EMA_PERIOD = 26

In [3]:
spy_data = pd.read_csv("SPY.csv")
spy_data.dropna(axis = 0)
spy_data.head()
spy_data = spy_data[0:500]
short_ema = spy_data.Close.ewm(span=SHORT_EMA_PERIOD,min_periods=0,adjust=False).mean()
long_ema = spy_data.Close.ewm(span=LONG_EMA_PERIOD,min_periods=0,adjust=False).mean()

NameError: name 'pd' is not defined

In [4]:
plt.plot(spy_data.Close)
plt.plot(spy_data.index, short_ema)
plt.plot(spy_data.index, long_ema)

NameError: name 'plt' is not defined